In [1]:
import kotlin.properties.Delegates
import kotlin.properties.ReadWriteProperty
import kotlin.reflect.KProperty
import java.time.Instant

In [2]:
class LazyHolder {
    val value: String by lazy {
        println(Instant.now().toString() + " inicjalizacja")
        "aaa"
    }
}

In [3]:
 val lazyHolder = LazyHolder()
 println(Instant.now().toString() + " stworzony obiekt")
 Thread.sleep(1000)
 println(lazyHolder.value)

2023-10-20T11:58:51.548959Z stworzony obiekt
2023-10-20T11:58:52.553892Z inicjalizacja
aaa


line 43 in Lazy.kt

In [4]:
class ObservableHolder(initial: Int) {

    var value: Int by Delegates.observable(initial) {
            property, oldValue, newValue -> println("${property.name} changed value from $oldValue to $newValue")
    }
}

In [5]:
val observableHolder = ObservableHolder(0)
observableHolder.value = 1
observableHolder.value += 10
observableHolder.value--

value changed value from 0 to 1
value changed value from 1 to 11
value changed value from 11 to 10


11

In [6]:
data class InvocationCounter(
    var getCounter: Int = 0,
    var setCounter: Int = 0
)

class CountInvocationDelegate<T>(private var value: T, private val counter: InvocationCounter): ReadWriteProperty<Any, T> {

    override fun getValue(thisRef: Any, property: KProperty<*>): T {
        counter.getCounter++
        return value
    }

    override fun setValue(thisRef: Any, property: KProperty<*>, value: T) {
        counter.setCounter++
        this.value = value
    }
}

class CustomDelegateHolder(initialValue: Int) {
    private val counter = InvocationCounter()
    var value: Int by CountInvocationDelegate(initialValue, counter)    

    // property delegation
    val getCount by counter::getCounter
    val setCount by counter::setCounter
}

In [7]:
val holder = CustomDelegateHolder(0)
holder.value = 1
holder.value = 2
holder.value = 5
holder.value = 4
println(holder.value)

println("property was set ${holder.setCount} times")
println("property was get ${holder.getCount} times")

4
property was set 4 times
property was get 1 times
